In [1]:
import numpy as np
import csv
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.layers import Conv2D, Dense, Dropout, Activation, Flatten, MaxPooling2D, LSTM
from keras.optimizers import SGD

/home/fx/env/stock/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
SET50 = ['ADVANC', 'AOT', 'BANPU', 'BBL', 'BCP', 'BDMS',
        'BEM', 'BH', 'BJC', 'BTS', 'CENTEL', 'CPALL', 
         'CPF', 'CPN', 'DTAC', 'EGCO', 'HMPRO', 
         'INTUCH', 'IRPC', 'KBANK', 'KCE', 'KKP', 'KTB', 'LH', 'MINT', 'PTT',
        'PTTEP', 'ROBINS', 'SCB', 'SCC', 
        'TCAP', 'TISCO', 'TMB', 'TOP', 'TRUE', 'TU']

In [37]:
lr = 0.1
epochs = 10000
batch_size = 100
optimizer = SGD(lr=lr)
activator = 'tanh'
loss_method = 'mse'
metrics = ['mae', 'accuracy']

In [38]:
def load_stock_data(symbol, year_start, year_end):
    stock_data = []
    with open('../Data set/FIXED_SET50/' + symbol + '.BK.csv', 'r') as csv_file:
        file_data = csv.reader(csv_file, delimiter=',')
        file_data = list(file_data)[1:]
        temp_value = 0
        for row in file_data:
            if row[1] is '':
                continue
            elif int(row[0][0:4]) >= year_start and int(row[0][0:4]) <= year_end:
                temp = float(row[1])
                if temp_value != 0:
                    unrealize = (temp - temp_value)/ temp_value
                else:
                    unrealize = 0
                stock_data.append([unrealize])
                temp_value = temp
                
    return stock_data

In [39]:
stock_datas = [ load_stock_data(x, 2008, 2014) for x in SET50 ]

rounds = len(stock_datas[0]) - 30

In [42]:
def create_cnn():
    model = Sequential()
    model.add(Conv2D(64, (7, 1), input_shape=(len(stock_datas), 30, 1), padding='same'))
    model.add(Activation(activator))
    model.add(MaxPooling2D(pool_size = (2, 1), data_format= "channels_last"))
    model.add(Flatten())
    return model

def create_model():
    cnn = create_cnn()
    model = Sequential()
    model.add(TimeDistributed(cnn, input_shape=(1, len(stock_datas), 30, 1)))
    model.add(LSTM(512, return_sequences=True))
    model.add(Activation(activator))
    model.add(Dropout(0.2))
    model.add(LSTM(512, return_sequences=True))
    model.add(Activation(activator))
    model.add(Dropout(0.2))
    model.add(LSTM(512))
    model.add(Activation(activator))
    model.add(Dropout(0.2))
    model.add(Dense(len(stock_datas)))
    model.summary()
    return model

In [43]:
model = create_model()
model.compile(loss=loss_method, optimizer=optimizer, metrics=metrics)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_6 (TimeDist (None, 1, 34560)          512       
_________________________________________________________________
lstm_12 (LSTM)               (None, 1, 512)            71829504  
_________________________________________________________________
activation_20 (Activation)   (None, 1, 512)            0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 1, 512)            0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 1, 512)            2099200   
_________________________________________________________________
activation_21 (Activation)   (None, 1, 512)            0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 1, 512)            0         
__________

In [44]:
data = []
count = 0
index = 0
while index < rounds:
    temp = [i[index : index + 30] for i in stock_datas]
    data.append([temp])
    index += 1
    
x = np.asarray(data)
x_train = x.astype('float32')

print(x_train.shape)

(1690, 1, 36, 30, 1) (1, 1, 36, 30, 1)


In [45]:
target = []
index = 0
while index < rounds:
    temp = [stock_datas[i][index + 30][0] for i in range(len(stock_datas))]
    target.append(temp)
    index += 1
    
y = np.asarray(target)
y_train = y.astype('float32')

print(y_train.shape)

(1690, 36) (725, 36)


In [46]:
model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size, verbose= 1)
# model.save('cnnlstmUnrealize.h5')

Epoch 1/10000
1690/1690 [==============================] - 5s 3ms/step - loss: 8.0857e-04 - mean_absolute_error: 0.0133 - acc: 0.0260
Epoch 2/10000
1690/1690 [==============================] - 1s 350us/step - loss: 8.0828e-04 - mean_absolute_error: 0.0133 - acc: 0.0302
Epoch 3/10000
1690/1690 [==============================] - 1s 352us/step - loss: 8.0811e-04 - mean_absolute_error: 0.0133 - acc: 0.0314
Epoch 4/10000
1690/1690 [==============================] - 1s 351us/step - loss: 8.0791e-04 - mean_absolute_error: 0.0134 - acc: 0.0325
Epoch 5/10000
1690/1690 [==============================] - 1s 354us/step - loss: 8.0777e-04 - mean_absolute_error: 0.0134 - acc: 0.0325
Epoch 6/10000
1690/1690 [==============================] - 1s 354us/step - loss: 8.0768e-04 - mean_absolute_error: 0.0134 - acc: 0.0325
Epoch 7/10000
1690/1690 [==============================] - 1s 353us/step - loss: 8.0751e-04 - mean_absolute_error: 0.0134 - acc: 0.0325
Epoch 8/10000
1690/1690 [=========================

1690/1690 [==============================] - 1s 364us/step - loss: 8.0711e-04 - mean_absolute_error: 0.0135 - acc: 0.0325
Epoch 120/10000
1690/1690 [==============================] - 1s 357us/step - loss: 8.0707e-04 - mean_absolute_error: 0.0135 - acc: 0.0325
Epoch 121/10000
1690/1690 [==============================] - 1s 357us/step - loss: 8.0713e-04 - mean_absolute_error: 0.0135 - acc: 0.0325
Epoch 122/10000
1690/1690 [==============================] - 1s 352us/step - loss: 8.0710e-04 - mean_absolute_error: 0.0135 - acc: 0.0325
Epoch 123/10000
1690/1690 [==============================] - 1s 352us/step - loss: 8.0709e-04 - mean_absolute_error: 0.0135 - acc: 0.0325
Epoch 124/10000
1690/1690 [==============================] - 1s 353us/step - loss: 8.0711e-04 - mean_absolute_error: 0.0135 - acc: 0.0325
Epoch 125/10000
 500/1690 [=======>......................] - ETA: 0s - loss: 8.9079e-04 - mean_absolute_error: 0.0136 - acc: 0.0440

KeyboardInterrupt: 